In [3]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
import torch.nn as nn
import torch.nn.functional as F

In [4]:
DATA_PATH = "../data/astro_dataset_maxia/astro_dataset_maxia"

In [5]:
image_sz = 128
batch_sz = 32

In [6]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((image_sz, image_sz)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

eval_transform = transforms.Compose([
    transforms.Resize((image_sz, image_sz)),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

In [7]:
train_data = datasets.ImageFolder(os.path.join(DATA_PATH, "training"), transform=train_transform)
test_data  = datasets.ImageFolder(os.path.join(DATA_PATH, "test"), transform=eval_transform)
val_data  = datasets.ImageFolder(os.path.join(DATA_PATH, "validation"), transform=eval_transform)

print("Classes:", train_data.classes)
print("Number of Classes:", len(train_data.classes))
print("Training Samples:", len(train_data))
print("Val Samples:", len(val_data))
print("Test Samples:", len(test_data))
print("Total Samples:", len(train_data)+len(val_data)+len(test_data))

Classes: ['asteroid', 'black_hole', 'earth', 'galaxy', 'jupiter', 'mars', 'mercury', 'neptune', 'pluto', 'saturn', 'uranus', 'venus']
Number of Classes: 12
Training Samples: 2416
Val Samples: 658
Test Samples: 345
Total Samples: 3419


In [8]:
class CNN(nn.Module):
    def __init__(self, num_classes, img_size=224):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, padding=1, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, padding=1, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, padding=1, kernel_size=3)

        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.25)

        feat_size = img_size//8
        flat_dim = 128*feat_size**2
        self.fc1 = nn.Linear(flat_dim, 256)
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)

        return x


In [9]:
train_loader = DataLoader(train_data, batch_size=batch_sz, shuffle=True)
val_loader   = DataLoader(val_data,   batch_size=batch_sz, shuffle=False)
test_loader  = DataLoader(test_data,  batch_size=batch_sz, shuffle=False)

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    model = CNN(len(train_data.classes), image_sz).to(device)

In [11]:
import torch.optim as optim

def train_cnn(lamb, num_epochs=10):
    model = CNN(num_classes=len(train_data.classes), img_size=image_sz).to(device)
    print(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=lamb)

    history = {"train_loss": [], "val_loss": [], "train_accuracy": [], "val_accuracy": []}

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * labels.size(0)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

        train_loss = running_loss / total
        train_accuracy = correct / total

        model.eval()
        val_loss_sum = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss_sum += loss.item() * labels.size(0)

                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()

        val_loss = val_loss_sum / val_total
        val_accuracy = val_correct / val_total

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_accuracy"].append(train_accuracy)
        history["val_accuracy"].append(val_accuracy)

        print(f"when lambda is {lamb} and epoch {epoch+1}/{num_epochs}, train accuracy is {train_accuracy} and val accuracy is {val_accuracy}")

    return model, history


In [ ]:
test_lamb = 1e-4
model_test, hist_test = train_cnn(lamb=test_lamb, num_epochs=3)


In [ ]:
lambs = [0, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3]

In [ ]:
res = {}

for lamb in lambs:
    model_lamb, hist_lamb = train_cnn(lamb=lamb, num_epochs=12)
    res[lamb] = hist_lamb
